In [ ]:
# 1.1 read data 
filename = 'adult.csv'
df = pd.read_csv(filename)
df.head(5)

In [ ]:
names = list(df.columns)
print(names)

['39', ' State-gov', ' 77516', ' Bachelors', ' 13', ' Never-married', ' Adm-clerical', ' Not-in-family', ' White', ' Male', ' 2174', ' 0', ' 40', ' United-States', ' <=50K']


In [ ]:
g = df[names[-1]].value_counts()
g.plot.bar()

In [ ]:
types = df.dtypes
print(types)

In [ ]:
from sklearn.preprocessing import LabelEncoder 
le = LabelEncoder()
for i in range(len(types)):
  if types[i]=='object':
    le.fit_transform(df[names[i]])
    df[names[i]] = le.transform(df[names[i]])

In [ ]:
data = df.values
print(data)

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(data)
data_scaled = scaler.transform(data)

dataset = [[None]]*len(data)
for i in range(len(data_scaled)):
  a = []
  for j in data_scaled[i]:
    a.append(j)
  dataset[i] = a
print(dataset[0])

[0.4520547945205479, 0.75, 0.04823759525135491, 0.6, 0.8, 0.3333333333333333, 0.2857142857142857, 0.0, 1.0, 1.0, 0.0, 0.0, 0.12244897959183672, 0.951219512195122, 0.0]


In [ ]:
from math import exp
from random import seed
from random import random
import pandas as pd
import numpy as np

# Initialize a network
def initialize_network(n_inputs, n_hidden, n_outputs):
	network = list()
	hidden_layer = [{'weights':[random() for i in range(n_inputs + 1)]} for i in range(n_hidden)]
	network.append(hidden_layer)
	output_layer = [{'weights':[random() for i in range(n_hidden + 1)]} for i in range(n_outputs)]
	network.append(output_layer)
	return network

# Calculate neuron activation for an input
def activate(weights, inputs):
	activation = weights[-1]
	for i in range(len(weights)-1):
		activation += weights[i] * inputs[i]
	return activation

# sigmoid_function neuron activation
def sigmoid_function(activation):
	return 1.0 / (1.0 + exp(-activation))

# Forward propagate input to a network output
def feedforward(network, row):
	inputs = row
	for layer in network:
		new_inputs = []
		for neuron in layer:
			activation = activate(neuron['weights'], inputs)
			neuron['output'] = sigmoid_function(activation)
			new_inputs.append(neuron['output'])
		inputs = new_inputs
	return inputs

# Calculate the derivative of an neuron output
def sigmoid_function_derivative(output):
	return output * (1.0 - output)

# Backpropagate
def backward_propagation(network, expected):
	for i in reversed(range(len(network))):
		layer = network[i]
		errors = list()
		if i != len(network)-1:
			for j in range(len(layer)):
				error = 0.0
				for neuron in network[i + 1]:
					error += (neuron['weights'][j] * neuron['delta'])
				errors.append(error)
		else:
			for j in range(len(layer)):
				neuron = layer[j]
				errors.append(neuron['output'] - expected[j])
		for j in range(len(layer)):
			neuron = layer[j]
			neuron['delta'] = errors[j] * sigmoid_function_derivative(neuron['output'])

# Gradient Descent
def update_weights(network, row, l_rate):
	for i in range(len(network)):
		inputs = row[:-1]
		if i != 0:
			inputs = [neuron['output'] for neuron in network[i - 1]]
		for neuron in network[i]:
			for j in range(len(inputs)):
				neuron['weights'][j] -= l_rate * neuron['delta'] * inputs[j]
			neuron['weights'][-1] -= l_rate * neuron['delta']

# Train the network
def train_network(network, train, l_rate, n_epoch, n_outputs):
	for epoch in range(n_epoch):
		sum_error = 0
		for row in train:
			outputs = feedforward(network, row)
			expected = [0 for i in range(n_outputs)]
			expected[int(row[-1])] = 1
			sum_error += sum([(expected[i]-outputs[i])**2 for i in range(len(expected))])
			backward_propagation(network, expected)
			update_weights(network, row, l_rate)
		print('>epoch=%d, lrate=%.3f, error=%.3f' % (epoch, l_rate, sum_error))




n_inputs = len(dataset[0]) - 1
n_outputs = len(set([row[-1] for row in dataset]))
network = initialize_network(n_inputs, 2, n_outputs)
train_network(network, dataset, 0.5, 20, n_outputs)
for layer in network:
	print(layer)

In [ ]:

def predict(network, row):
	outputs = feedforward(network, row)
	return outputs.index(max(outputs))

In [ ]:
Correct = 0
for row in dataset:
 prediction = predict(network, row)

 if(row[-1]==prediction):
   Correct+=1
print("Accuracy = %f" % (Correct/len(dataset)))


Accuracy= 0.833323
